In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
prefix = 'hf-community-event-project'

In [ ]:
import os
import boto3

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix,"data",'train.csv')).upload_file('data/train.csv')


In [ ]:
from sagemaker.huggingface import HuggingFace

huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    instance_type='ml.p3.2xlarge',
    instance_count = 1,
    role = role,
    transformers_version='4.12',
    tensorflow_version='2.5',
    py_version='py37',
    hyperparameters={
        'sampling': 0, # take the whole dataset
        "epochs": 1
    }
)

In [ ]:
huggingface_estimator.fit({
    'train': 's3://{}/{}/data'.format(bucket,prefix)
})

# Deploy the model
Once the training has completed, you can deploy the model in Sagemaker!

In [ ]:
predictor = huggingface_estimator.deploy(
    initial_instance_count = 1,
    instance_type = "ml.m5.xlarge")

## Prediction example
Once the model is deployed, we can use it!

predictor.predict('This is not a very good comment')

## Delete endpoint 
Finally, to avoid being charged, please delete your endpoint    

In [ ]:
predictor.delete_endpoint()